## Colocando os Conhecimentos em Prática

Na aula de hoje, nós revisitaremos a tarefa de classificação de litologia no dataset FORCE utilizando as técnicas aprendidas ao longo da semana.

A partir do que aprendemos na última semana, sabemos que temos três grandes problemas que precisamos abordar a fim de melhorar os resultados de nossa classificação:

- A rede deve ter uma capacidade maior em comparação com a usada na semana 01;
- Devemos tratar do grande desbalanceamento das classes em nossos dados, que fez com que nossa rede inicial prevesse majoritariamente algumas poucas classes.
- Devemos implementar pensar em maneiras de diminuir um possível overfitting, principalmente ao aumentarmos a capacidade de nosso modelo.

Agora é com você! Construa um Multi Layer Perceptron, usando pytorch, para mapear cada ponto do poço para a litologia correta.

> Dica: Considere fazer suas predições em uma janela deslizante.

Abaixo está um passo a passo do que deve ser feito.

1. Leia os dados baixados e extraídos
> Dica: use a biblioteca pandas. Exemplo: pd.read_csv("train.csv", sep=';')
2. Divida os poços em janelas da profundidade que quiser.
3. Construa sua base de treino e teste, bem como o dataloder para elas.
4. Construa seu MLP usando pytorch.
5. Treine o modelo.
6. Extraia as previsões em seu dado de teste para cada janela, em junte-as para formar a segmentação completa do poço.
7. Meça a qualidade de seu algoritmo com métricas a sua escolha.
8. Teste diferentes otimizadores, técnicas de regularização.



In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torchmetrics.functional import accuracy
from collections import Counter
from tqdm import tqdm

In [2]:
# Lendo os dados

data_dir = '/pgeoprj2/ciag2024/dados/FORCE'

df_train = pd.read_csv(f"{data_dir}/train.csv", sep=';')
df_test = pd.read_csv(f"{data_dir}/leaderboard_test_features.csv", sep=';')
df_test_target = pd.read_csv(f"{data_dir}/leaderboard_test_target.csv", sep=';')
df_test = pd.merge(df_test, df_test_target, on=["WELL", "DEPTH_MD"], how="inner")

In [3]:
df_train.head()

,WELL,DEPTH_MD,X_LOC,Y_LOC,Z_LOC,GROUP,FORMATION,CALI,RSHA,RMED,...,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO,FORCE_2020_LITHOFACIES_LITHOLOGY,FORCE_2020_LITHOFACIES_CONFIDENCE
0,15/9-13,494.528,437641.96875,6470972.5,-469.501831,NORDLAND GP.,NaN,19.480835,NaN,1.611410,...,34.636410,NaN,NaN,-0.574928,NaN,NaN,NaN,NaN,65000,1.0
1,15/9-13,494.680,437641.96875,6470972.5,-469.653809,NORDLAND GP.,NaN,19.468800,NaN,1.618070,...,34.636410,NaN,NaN,-0.570188,NaN,NaN,NaN,NaN,65000,1.0
2,15/9-13,494.832,437641.96875,6470972.5,-469.805786,NORDLAND GP.,NaN,19.468800,NaN,1.626459,...,34.779556,NaN,NaN,-0.574245,NaN,NaN,NaN,NaN,65000,1.0
3,15/9-13,494.984,437641.96875,6470972.5,-469.957794,NORDLAND GP.,NaN,19.459282,NaN,1.621594,...,39.965164,NaN,NaN,-0.586315,NaN,NaN,NaN,NaN,65000,1.0
4,15/9-13,495.136,437641.96875,6470972.5,-470.109772,NORDLAND GP.,NaN,19.453100,NaN,1.602679,...,57.483765,NaN,NaN,-0.597914,NaN,NaN,NaN,NaN,65000,1.0


In [4]:
# Nomes das litologias
lithology_keys = {
    30000: 'Sandstone',
    65030: 'Sandstone/Shale',
    65000: 'Shale',
    80000: 'Marl',
    74000: 'Dolomite',
    70000: 'Limestone',
    70032: 'Chalk',
    88000: 'Halite',
    86000: 'Anhydrite',
    99000: 'Tuff',
    90000: 'Coal',
    93000: 'Basement'
}

target_col = "FORCE_2020_LITHOFACIES_LITHOLOGY"
feature_cols = ["GR", "RHOB", "NPHI", "PEF", "DTC", "RDEP", "RSHA", "RMED"] # Escolha as features que quiser

# Removendo valores faltantes
df_train = df_train.dropna(subset=feature_cols+[target_col]).reset_index(drop=True)
df_test = df_test.dropna(subset=feature_cols+[target_col]).reset_index(drop=True)

# Codificando as labels para que as possamos passar para o modelo
le = LabelEncoder()
all_possible_classes = list(lithology_keys.keys())
le.fit(all_possible_classes)
df_train[target_col] = le.transform(df_train[target_col])
df_test[target_col] = le.transform(df_test[target_col])

# Normalizando as featues
scaler = StandardScaler()
df_train[feature_cols] = scaler.fit_transform(df_train[feature_cols])
df_test[feature_cols] = scaler.transform(df_test[feature_cols])

In [5]:
# Aqui estamos listando todos os poços presentes no dataset de treino
wells = df_train["WELL"].unique()
print(wells)

['15/9-13' '15/9-15' '15/9-17' '16/10-3' '16/10-5' '16/4-1' '16/7-5'
 '25/11-19 S' '25/2-13 T4' '25/2-14' '25/2-7' '25/3-1' '25/4-5' '25/5-1'
 '25/5-4' '25/8-5 S' '25/8-7' '25/9-1' '26/4-1' '29/6-1' '30/6-5'
 '31/2-19 S' '31/2-7' '31/2-8' '31/2-9' '31/3-2' '31/3-4' '31/4-5'
 '32/2-1' '33/9-17' '34/10-21' '34/10-35' '34/11-2 S' '34/12-1' '34/2-4'
 '34/7-21' '34/8-1' '35/11-1' '35/11-6' '35/11-7' '35/12-1' '35/8-4'
 '7/1-2 S']


In [6]:
df_train["WELL"].value_counts()

WELL
33/9-17       17337
31/2-19 S     16308
34/10-21      13945
15/9-15       13290
15/9-17       12873
26/4-1        12824
25/2-7        12043
16/7-5        10266
34/12-1        9109
35/11-6        8378
34/7-21        7576
25/8-5 S       7455
35/11-7        7227
35/12-1        7073
25/11-19 S     6908
31/2-8         6582
35/11-1        6195
25/3-1         6010
31/3-4         5223
34/11-2 S      5073
16/10-3        4578
25/2-14        3845
34/8-1         3735
25/9-1         3596
34/2-4         3588
29/6-1         3409
25/5-1         2954
32/2-1         2789
30/6-5         2572
16/10-5        2498
25/8-7         2452
35/8-4         2263
31/4-5         2046
25/4-5         1891
31/2-9         1787
15/9-13        1623
25/2-13 T4     1574
7/1-2 S        1530
25/5-4         1328
34/10-35       1066
31/2-7         1008
31/3-2          620
16/4-1          615
Name: count, dtype: int64

In [38]:
# Criando janelas nos dados
# Optimized function for creating windows
def create_windows(data, features, target, window_size=10):
    X = np.lib.stride_tricks.sliding_window_view(data[features].values, (window_size, len(features)))[:-1, :, :]
    Y = data[target].iloc[window_size:].values
    return X.squeeze(), Y

window_size = 10 # Escolha o tamanho de janela que quiser
X_train = []
y_train = []
for well in wells:
    well_data = df_train[df_train['WELL'] == well]
       
    x_well, y_well = create_windows(well_data, feature_cols, target_col, window_size=window_size)
    X_train.append(x_well)
    y_train.append(y_well)

X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)
X_test, y_test = create_windows(df_test, feature_cols, target_col, window_size=window_size)

print(f"X_train shape: {X_train.shape} || X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape} || y_test shape: {y_test.shape}")

X_train shape: (244632, 10, 8) || X_test shape: (26583, 10, 8)
y_train shape: (244632,) || y_test shape: (26583,)


In [39]:
# Criando o dataset que utilizaremos
class WellDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# definindo os datasets
train_dataset = WellDataset(X_train, y_train)
test_dataset = WellDataset(X_test, y_test)

# definindo os dataloaders
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
test_loader = DataLoader(test_dataset, batch_size=8)